### **Cài đặt những thư viện cần thiết**

In [ ]:
!pip install langchain langchain-community langchain-huggingface chromadb pymupdf transformers accelerate torch faiss-cpu networkx

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.llms import Ollama
import torch
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from langchain.schema import Document
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
loader = PyMuPDFLoader("/content/doc1.pdf")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)
chunked_documents = text_splitter.split_documents(documents)
db1 = Chroma.from_documents(chunked_documents, embeddings)

In [ ]:
loader = PyMuPDFLoader("/content/doc2.pdf")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)
chunked_documents = text_splitter.split_documents(documents)
db2 = Chroma.from_documents(chunked_documents, embeddings)

In [ ]:
loader = PyMuPDFLoader("/content/doc3.pdf")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)
chunked_documents = text_splitter.split_documents(documents)
db3 = Chroma.from_documents(chunked_documents, embeddings)

In [ ]:
login("HUGGING FACE TOKEN")
def load_llm(model_file):
    # Load model và tokenizer
    model = AutoModelForCausalLM.from_pretrained(
        model_file,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True
    )

    tokenizer = AutoTokenizer.from_pretrained(
        model_file,
        trust_remote_code=True
    )



    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=5000,
        temperature=0.1,
        device_map="auto"
    )

    llm = HuggingFacePipeline(pipeline=pipe)

    return llm

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

retriever1 = db1.as_retriever(search_kwargs={"k": 4})
retriever2 = db2.as_retriever(search_kwargs={"k": 4})
retriever3 = db3.as_retriever(search_kwargs={"k": 4})

llm = load_llm("meta-llama/Llama-3.2-3B-Instruct")

rag_prompt = PromptTemplate.from_template(
  """
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an AI assistant helping to answer questions based on provided contexts

    <|eot_id|><|start_header_id|>user<|end_header_id|>

    Context: {context}
    Question: {input}

    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
  """
)


rewrite_prompt = PromptTemplate.from_template(
  """
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an AI assistant helping to ewrite the given answer to make it cleaner, more structured, and easier to understand.

    <|eot_id|><|start_header_id|>user<|end_header_id|>

    answer: {input}

    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
  """
)


rag_chain = LLMChain(llm=llm, prompt=rag_prompt)
rewrite_chain = LLMChain(llm=llm, prompt=rewrite_prompt)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-6-9e02f5efc5ae>:27: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
<ipython-input-7-fc0fb791db61>:42: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  rag_chain = LLMChain(llm=llm, prompt=rag_prompt)


In [ ]:
import torch
import re

# torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=None, abbreviated=False)

def extract_answer(text):
    pattern = r"<\|eot_id\|><\|start_header_id\|>assistant<\|end_header_id\|>(.*)"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()


def query_rag_with_llm(rag_chain, rewrite_chain, question):
    retrieved_docs = retriever1.get_relevant_documents(question)
    retrieved_docs = [doc.page_content for doc in retrieved_docs]
    result = rag_chain.invoke({"input": question, "context": retrieved_docs})


    retrieved_docs = retriever2.get_relevant_documents(question)
    retrieved_docs = [doc.page_content for doc in retrieved_docs]
    retrieved_docs.append(extract_answer(result['text']))
    result = rag_chain.invoke({"input": question, "context": retrieved_docs})


    retrieved_docs = retriever3.get_relevant_documents(question)
    retrieved_docs = [doc.page_content for doc in retrieved_docs]
    retrieved_docs.append(extract_answer(result['text']))
    result = rag_chain.invoke({"input": question, "context": retrieved_docs})


    result = rewrite_chain.invoke({"input": result})
    return extract_answer(result['text'])

question = "What is Binary Search Trees?"
print(query_rag_with_llm(rag_chain, rewrite_chain, question))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


\n   Binary Search Trees (BSTs) are a type of binary tree where each node has a key and a value, and satisfies the restriction that the key in any node is larger than the keys in all nodes in that node\'s left subtree and smaller than the keys in all nodes in that node\'s right subtree.'}

    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
   Here is a rewritten version of the text with improvements in structure, clarity, and readability:

**What are Binary Search Trees?**

Binary Search Trees (BSTs) are a type of binary tree that stores data in a way that allows for efficient searching. Each node in a BST has a key and a value, and the key in any node is larger than the keys in all nodes in its left subtree and smaller than the keys in all nodes in its right subtree.

**Key Characteristics of Binary Search Trees**

* Each node has a key and a value.
* The key in any node is larger than the keys in all nodes in its left subtree.
* The key in any node is smaller than the keys i

In [ ]:
print(answer)

In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok transformers

In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import conf, ngrok
from fastapi.middleware.cors import CORSMiddleware

# Bật lại event loop trong notebook
nest_asyncio.apply()

# Ngrok token
conf.get_default().auth_token = "2vbdTSzzDq8tAiZlh3ai4kEvZ5d_348zcse9Lii6TfWJagWpo"

# Khởi tạo FastAPI
app = FastAPI()

# Cho phép CORS để web gửi request được
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# Dữ liệu gửi đến
class Message(BaseModel):
    message: str

# Định nghĩa endpoint
@app.post("/chat")
def chat_api(msg: Message):
    prompt = msg.message
    print(prompt)
    output = query_rag_with_llm(rag_chain, prompt)  # Đã định nghĩa sẵn
    return {"reply": output}

# Tạo tunnel
public_url = ngrok.connect(8000)
print(f"🔗 API endpoint: {public_url}/chat")

# Chạy server
uvicorn.run(app, port=8000)
